## Assumptions and Constants
Let's use th metal ABR core because it sounds, well, metal.

In [1]:
# Reactor constants
beta_eff = 0.00334
prompt_lifetime = 0.38

Na void positive reactivity due to spectral hardening

Less fast neutrons, less absorption (?)

![image](img/Pu_239_cross_section.png)

## Operative word is "Local"

* Void worth $4.44 assuming "flowing sodium completely voided in ALL active and above-core regions" ([Argonne's Fast Reactor Physics - 2](https://www.nrc.gov/docs/ML1914/ML19148A801.pdf))
* How to apply this for local voiding?
* Ultimately want reactivty as a function of sodium density, i.e. $k(\rho_{Na})$

## Driven by scattering out of fast spectrum
Assumptions to derive $k(\rho_{Na})$
* Assume two group equation
* Infinite reactor (or perfect reflector)
* All relevant fissions occur in fast spectrum (terrible assumption...)
* Everything scattered out of fast spectrum is absorbed (also pretty bad)

## Two group equation
Group 1 - fast spectrum

Group 2 - everything else
$$
\frac{1}{k}\nu_1 \Sigma_f \phi_1 - \Sigma_{a1} \phi_1 - \Sigma_{1\rightarrow 2}\phi_1 = 0 \\
\Sigma_{1 \rightarrow 2} \phi_1 - \Sigma_{a2} \phi_2  = 0
$$

Solving for k in the group 1 equation
$$
k = \frac{\nu_1 \Sigma_{f1}}{\Sigma_{a1} + \Sigma_{1 \rightarrow 2}} 
$$

Using group 2 to make a statement about $\Sigma_{a1}$, noting that $\Sigma_{a2} \propto \Sigma_{a1}$
$$
\Sigma_{a2}\phi_2 = \Sigma_{1 \rightarrow 2} \phi_1 = C_a \Sigma_{a1}\phi1\\
\Sigma_{a1} = \frac{\Sigma_{1 \rightarrow 2} \phi_1}{C_a \phi_2}
$$
where $C_a$ is a proportionality constant.

Substituting $\Sigma_{a1}$ into the expression for k

$$
k = \frac{\nu_1 \Sigma_{f1}}{\frac{\Sigma_{1 \rightarrow 2} \phi_1}{C_a \phi_2} + \Sigma_{1 \rightarrow 2}}
= \frac{\nu_1 \Sigma_{f1}}{\Sigma_{1 \rightarrow 2}[\frac{\phi_1}{C_a \phi_2} + 1]}

= \frac{\nu_1\Sigma_{f1}}{\frac{\rho_{Na}A_v}{M_{Na}}\sigma_{1 \rightarrow 2}[\frac{\phi_1}{C_a \phi_2} + 1]}
$$


$$
k = \frac{\nu_1 \Sigma_{f1}}{\Sigma_{a1} + \Sigma_{1 \rightarrow 2}}

= \frac{\nu_1\Sigma_{f1}}{\Sigma_{a1} + \frac{\rho_{Na}A_v}{M_{Na}}\sigma_{1 \rightarrow 2}}

= \frac{A}{B + C\rho_{Na}}
$$



$$
k(\Sigma_{a1} + \Sigma_{1 \rightarrow 2}) = \nu_1 \Sigma_{f1}
$$
$$
k\left(\frac{\Sigma_{a1}}{\nu_1 \Sigma_{f1}} + \frac{\Sigma_{1 \rightarrow 2}}{\nu_1 \Sigma_{f1}}\right) = 1
$$
$$
k\left(\frac{\Sigma_{a1}}{\nu_1 \Sigma_{f1}} + \frac{\rho_{Na}A_v}{M_{Na}}\frac{\sigma_{1 \rightarrow 2}}{\nu_1 \Sigma_{f1}}\right) = 1
$$
$$
k\left(A + B\rho_{Na}\right) = 1
$$


We have two known reactor conditions that we can now use to solve for A and B in the above equation:
1. Void worth when entire core is voided ($4.44 BOC, metal, startup)
2. Assume k = 1 when entire core is liquid

For Na density, we will use (from [ANL-RE-95-2](https://www.ne.anl.gov/eda/ANL-RE-95-2.pdf))
1. Density of vapor sodium at saturation ($\sim 1200 K$ at operating pressure from [Figure III.7-12](https://publications.anl.gov/anlpubs/2017/04/134264.pdf)) is $0.394 \frac{kg}{m^3}$.
2. Density of liquid sodium at core average temperature $700 K$ (average of $350^o C$ cold pool temperature and $500^o C$ hot temperature) is $852 \frac{kg}{m^3}$.

This results in a 2x2 linear system that we can use to solve for A and B:
$$
Ak_1 + Bk_1\rho_1 = 1 \\
Ak_2 + Bk_2\rho_2 = 1
$$

$$
\begin{bmatrix}
k_1 & k_1 \rho_1 \\
k_2 & k_2 \rho_2 \\
\end{bmatrix}

\begin{bmatrix}
A \\ B
\end{bmatrix} 

=

\begin{bmatrix}
1 \\ 1
\end{bmatrix}
$$

It's a little confusing, but don't forget $\rho$ in these equations is Na density, not reactivity. Now for some python! (Finally...)

In [ ]:
"""A few helper functions before we get started.

And a general note on comments since you'll want to know how I
work. My rule of thumb is to write comments according to my audience.
In many cases, my audience is engineers, and therefore my comments
are extensive. But comments, like lines of code, represent inventory
of an application. As your inventory goes up, your maintenance costs
increase as well. Comments require maintenance just like source code
in the case of refactoring. If the code is obvious or the audience is
savvy, comments oftentimes are not necessary. However, I'll be using
lots of comments in this project assuming my audience is primarily engineers.
Also computational code is confusing and hard to read.
"""
import pint

# I've had success using pint to keep me from making dumb unit errors.
# So I'll try not to embarass myself in front of yall too badly.
ureg = pint.UnitRegistry()
Quant = pint.Quantity

def round_quant(quantity, sig_fig=4):
    """Return pint quantity reduced to given significant digits as a string."""
    value = round(
        quantity.m, 
        sig_fig-int(floor(log10(abs(quantity.m)))) - 1
    )
    return str(value) + ' ' + str(quantity.units)

# A few ABR constants assuming metal core and BOC we'll need
void_worth = 4.44 # Dollars
beta_eff = 0.0034 # Delayed neutron fraction for metal core

# Generating coefficients for the 2x2 system for k(rho_na)
# k1 is solved from the void worth
void_reactivity = void_worth * beta_eff
k_1 = 1 / (1 - void_reactivity)
# Solving the 2x2 system to 

np.linalg.solve

Assuming all other values are independent of $\rho_{Na}$ (ignoring absorption), we can simplify this to
$$
k = \frac{C}{\rho_{Na}}
$$
A big leap I know, but we get the behavior: $\rho_{Na}\downarrow$ leads to $k \uparrow$, which makes my tummy feel a little better moving forward with this assumption.

To solve for C, we'll use the $4.44 number from above. NOW for some python (finally)

In [13]:
# ABR constants assuming metal core and BOC
void_worth = 4.44 # Dollars
beta_eff = 0.0034 # Delayed neutron fraction for metal core

# Reactivity as a function of worth (in dollars)
reactivity = void_worth * beta_eff

# Expression for multiplication factor in terms of reactivity
k = 1 / (1-reactivity)
print ("Whole core void multiplication factor: ", '%.4g' % k)

Whole core void multiplication factor:  1.015


Density of sodium at saturation ($\sim 1100^o C$ atmospheric pressure) is $0.168 \frac{kg}{m^3}$. Solving for C:

In [33]:
import pint

# I've had success using pint to keep me from making dumb unit errors.
# So I'll try not to embarass myself in front of yall too badly.
ureg = pint.UnitRegistry()
Quant = pint.Quantity

def round_quant(quantity, sig_fig=4):
    """Return pint quantity reduced to given significant digits as a string."""
    value = round(
        quantity.m, 
        sig_fig-int(floor(log10(abs(quantity.m)))) - 1
    )
    return str(value) + ' ' + str(quantity.units)


In [34]:
# Solving for C to get our density factor
rho_na_vapor = Quant(0.168, 'kg/m^3')
C = k*rho_na_vapor

print(
    "Multiplaction factor ratio as a function of sodium density: ",
    round_quant(C)
)


Multiplaction factor ratio as a function of sodium density:  0.1706 kilogram / meter ** 3


Now we can return reactivity as a function of the amount of voided sodium inside of one assembly. 
This is my interpretation of "Local" for the project.
A subchannel code (e.g. NUPAC) could be used to generate this total void fraction. As it is I've spent a lot of time on this miniproject and won't try and build a subchannel solver right now.

In [ ]:
def local_reactivity(void_fraction):
    """Calculate the local reactivity of an assembly.
    
    Using the void worth of the active whole-core, calculate the
    reactivity insertion from a fraction of void within a single
    assembly using the following relationships:
    rho = k - 1 / k
    k = C / rho_na
    rho_na = (Mass of liquid + mass of vapor) / Total core volume

    Args:
        void_fraction (float): Fraction of a single assembly that is 
        vapor.
        1.0 = Entire assembly is vapor
        0.0 = Enteire assembly is liquid

    Returns:
        float: Local reactivity insertion from the local voiding of a
        single assembly.
    """
    


To get an expression for $\rho_{Na}$ we'll need some geomtry information, and more assumptions of course.

In [35]:
import pint
import numpy as np
from math import log10, floor

# I've had success using pint to keep me from making dumb unit errors.
# So I'll try not to embarass myself in front of yall too badly.
ureg = pint.UnitRegistry()

# Calculate the total volume of an assembly for 217 hexagonal assembly
# with 8 pins along each side, only considering the active core region
pins_per_side = 8
pitch = 16.142 * ureg.cm
assembly_side = pitch
active_core_height = 81.3 * ureg.cm
assembly_volume = float(3*np.sqrt(3)/2)*assembly_side**2 * active_core_height

# Calculate the total volume of fuel pins in an assembly
total_pins = 217
pin_diameter = 0.755 * ureg.cm
pin_volume = 3.1415 * (pin_diameter/2)**2 * active_core_height
total_pin_volume = pin_volume * total_pins

# Calculate the sodium volume in the active core region
sodium_volume = assembly_volume - total_pin_volume

print(
    "Assembly volume in active core region: ", 
    round_quant(assembly_volume, 4)
)
print(
    "Total pin volume in active core region: ", 
    round_quant(total_pin_volume, 4)
)
print(
    "Sodium volume in active core region: ", 
    round_quant(sodium_volume, 4)
)

Assembly volume in active core region:  55040.0 centimeter ** 3
Total pin volume in active core region:  7898.0 centimeter ** 3
Sodium volume in active core region:  47140.0 centimeter ** 3
